In [248]:
#Imports needed

import quandl
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime

In [249]:
# Setting the quandl API key variable within the quandl class
quandl.ApiConfig.api_key = os.getenv("QUANDL_API_KEY")

#getting the Sentiment Data from quandl
sentiment_data = pd.DataFrame(quandl.get(["AAII/AAII_SENTIMENT.1", "AAII/AAII_SENTIMENT.2", "AAII/AAII_SENTIMENT.3"]))#, start_date = "1987-07-24", end_date = "2020-01-20")#['Date', 'Bullish', 'Neutral', 'Bearish']
sentiment_data.columns = ['Bullish', 'Neutral', 'Bearish']
sentiment_data.reset_index(inplace = True)
sentiment_data.head()

,Date,Bullish,Neutral,Bearish
0,1987-07-24,0.36,0.50,0.14
1,1987-07-31,0.26,0.48,0.26
2,1987-08-07,0.56,0.15,0.29
3,1987-08-14,0.45,0.35,0.20
4,1987-08-21,0.66,0.28,0.06


In [308]:
sentiment_data['Bullish'] = sentiment_data['Bullish'].astype(float)
sentiment_data['Neutral'] = sentiment_data['Neutral'].astype(float)
sentiment_data['Bearish'] = sentiment_data['Bearish'].astype(float)
sentiment_data.dtypes

Date       datetime64[ns]
Bullish           float64
Neutral           float64
Bearish           float64
dtype: object

In [251]:
#Reading S&P500 Data
csvpath = Path("SP500.csv")
sp500_data = pd.read_csv(csvpath)
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
sp500_data['Close'] = sp500_data['Close'].astype(float)
sp500_data.set_index('Date', drop = True, inplace = True)
sp500_data.head()

,Close
Date,
1987-07-24,309.27
1987-07-27,310.65
1987-07-28,312.33
1987-07-29,315.65
1987-07-30,318.05


In [299]:
#next_day_1 = sentiment_data['Date'][0] + pd.DateOffset(days=1)
#next_day_2 = sentiment_data['Date'][0] + pd.DateOffset(days=2)
#next_day_3 = sentiment_data['Date'][0] + pd.DateOffset(days=3)
#print(next_day_1)
#print(next_day_2)


In [306]:
#Calculating S&P500 Daily changes in % (1 Day)
sp500_data['%Change'] = sp500_data.pct_change(1)
#sp500_data['AUX'] = ''
#sp500_data['AUX'][0]='1000'
sp500_data.head(10)

,Close,%Change
Date,,
1987-07-24,309.27,NaN
1987-07-27,310.65,0.004462
1987-07-28,312.33,0.005408
1987-07-29,315.65,0.010630
1987-07-30,318.05,0.007603
1987-07-31,318.66,0.001918
1987-08-03,317.57,-0.003421
1987-08-04,316.23,-0.004220
1987-08-05,318.45,0.007020


In [296]:
i=0
consolidated_dict = {}
consolidated_dict = {'Date':['T+1','T+2','T+3','T+4']}
for i in range(len(sentiment_data)-2):
    aux_df = sp500_data.loc[sentiment_data['Date'][i]:sentiment_data['Date'][i+2]].reset_index()
    #print(aux_df)
    #consolidated_dict[str(sentiment_data['Date'][i])] = [str(aux_df['Date'][1]), str(aux_df['Date'][2]), str(aux_df['Date'][3]), str(aux_df['Date'][4])]
    consolidated_dict[str(sentiment_data['Date'][i])] = [aux_df['%Change'][1], aux_df['%Change'][2], aux_df['%Change'][3], aux_df['%Change'][4]]
    i+=1

aux_df = pd.DataFrame.from_dict(consolidated_dict, orient='index')

aux_df.reset_index(inplace=True)
aux_df.columns = ['Date', 'T+1','T+2','T+3','T+4']
aux_df.drop([0], inplace=True)
aux_df['Date'] = pd.DatetimeIndex(aux_df['Date'])
aux_df.head()

,Date,T+1,T+2,T+3,T+4
1,1987-07-24,0.00446212,0.00540802,0.0106298,0.00760336
2,1987-07-31,-0.00342057,-0.00421954,0.00702021,0.0114304
3,1987-08-07,0.0154799,0.01625,-0.00282003,0.00679924
4,1987-08-14,0.000359292,-0.0145461,0.00176158,0.0151896
5,1987-08-21,-0.00765109,0.0103201,-0.00653265,-0.00953463


In [297]:
# Concatenate sentiment_data & aux_df DataFrames into a single DataFrame
sentiment_data.set_index(sentiment_data['Date'], inplace = True)
aux_df.set_index(aux_df['Date'], inplace=True)
joined_data_cols = pd.concat([sentiment_data, aux_df], axis='columns', join='inner')
joined_data_cols.drop(columns = 'Date',inplace = True)
joined_data_cols.tail()

,Bullish,Neutral,Bearish,T+1,T+2,T+3,T+4
Date,,,,,,,
2019-12-05,0.317152,0.391586,0.291262,0.00913573,-0.00316284,-0.00109695,0.0029082
2019-12-12,0.376316,0.363158,0.260526,7.25879e-05,0.00714782,0.000335271,-0.00043226
2019-12-19,0.440945,0.354331,0.204724,0.00494483,0.000866131,-0.000195409,0.00512816
2019-12-26,0.418879,0.365782,0.215339,3.39516e-05,-0.00578083,0.00294602,0.00837878
2020-01-02,0.372159,0.409091,0.218750,-0.00705987,0.00353339,-0.00280321,0.00490242


In [300]:
joined_data_cols['Bullish'] = joined_data_cols['Bullish'].astype(float)
joined_data_cols['Neutral'] = joined_data_cols['Neutral'].astype(float)
joined_data_cols['Bearish'] = joined_data_cols['Bearish'].astype(float)
joined_data_cols['T+1'] = joined_data_cols['T+1'].astype(float)
joined_data_cols['T+2'] = joined_data_cols['T+2'].astype(float)
joined_data_cols['T+3'] = joined_data_cols['T+3'].astype(float)
joined_data_cols['T+4'] = joined_data_cols['T+4'].astype(float)
joined_data_cols.dtypes

Bullish    float64
Neutral    float64
Bearish    float64
T+1        float64
T+2        float64
T+3        float64
T+4        float64
dtype: object

In [311]:
joined_data_cols['Bu<->Be'] = joined_data_cols[['Bullish', 'Bearish']].max(axis=1)-joined_data_cols[['Bullish', 'Bearish']].min(axis=1)

In [312]:
joined_data_cols

,Bullish,Neutral,Bearish,T+1,T+2,T+3,T+4,Bu<->Be
Date,,,,,,,,
1987-07-24,0.360000,0.500000,0.140000,0.004462,0.005408,0.010630,0.007603,0.220000
1987-07-31,0.260000,0.480000,0.260000,-0.003421,-0.004220,0.007020,0.011430,0.000000
1987-08-07,0.560000,0.150000,0.290000,0.015480,0.016250,-0.002820,0.006799,0.270000
1987-08-14,0.450000,0.350000,0.200000,0.000359,-0.014546,0.001762,0.015190,0.250000
1987-08-21,0.660000,0.280000,0.060000,-0.007651,0.010320,-0.006533,-0.009535,0.600000
...,...,...,...,...,...,...,...,...
2019-12-05,0.317152,0.391586,0.291262,0.009136,-0.003163,-0.001097,0.002908,0.025890
2019-12-12,0.376316,0.363158,0.260526,0.000073,0.007148,0.000335,-0.000432,0.115790
2019-12-19,0.440945,0.354331,0.204724,0.004945,0.000866,-0.000195,0.005128,0.236221
